# LLM-as-a-Judge Tutorial: Tool Selection Evaluation with MLflow

This interactive notebook demonstrates how to use MLflow's LLM-as-a-Judge pattern to evaluate AI agent decisions.

## Tutorial Goals

1. Use MLflow tracing to capture agent actions
2. Create a judge using `mlflow.genai.judges.make_judge()`
3. Evaluate agent decisions using the judge
4. Integrate with MLflow experiments for reproducibility

## Scenario

An AI agent selects a tool to answer a user query. The judge evaluates whether the agent chose the appropriate tool.

**Evaluation Criteria:**
- Does the selected tool match the user's intent?
- Can this tool address the task requirements?
- Are there more suitable tools available?

---

Based on: [Using LLM as a Judge](https://medium.com/@juanc.olamendy/using-llm-as-a-judge-to-evaluate-agent-outputs-a-comprehensive-tutorial-00b6f1f356cc)

## Setup: Import Dependencies

First, let's import all the necessary libraries.

In [1]:
from genai.common import get_client
from genai.common.config import AgentConfig
from genai.agents.tools_selection.prompts import get_judge_instructions, get_tool_selection_prompt
import mlflow
from mlflow.entities import SpanType
from mlflow.genai.judges import make_judge
from typing import Dict, Any, List
from typing_extensions import Literal
import os
from pathlib import Path

# Load environment variables from .env file if it exists
try:
    from dotenv import load_dotenv
    env_file = Path(".env")
    if env_file.exists():
        load_dotenv(env_file)
        print(f"✓ Loaded environment variables from {env_file.absolute()}")
    else:
        print(f"ℹ️  No .env file found at {env_file.absolute()}")
        print("   You can create one with your credentials or set environment variables manually")
except ImportError:
    print("ℹ️  python-dotenv not installed. Install with: pip install python-dotenv")
    print("   Or set environment variables manually in the cells below")

✓ Loaded environment variables from /Users/jules/git-repos/mlflow-misc/genai/agents/tools_selection/.env


## Configuration: Set Up Environment

**Two ways to configure credentials:**

1. **Recommended**: Create a `.env` file in this directory with your credentials
2. **Alternative**: Uncomment and set credentials in the cells below

### Create a `.env` file (Recommended)

Create a file named `.env` in the same directory as this notebook:

**For Databricks:**
```
DATABRICKS_TOKEN=your-token-here
DATABRICKS_HOST=https://your-workspace.cloud.databricks.com
```

**For OpenAI:**
```
OPENAI_API_KEY=sk-your-key-here
```

The cell above will automatically load these credentials.

In [2]:
# ============================================================================
# CONFIGURATION: Choose your provider and models
# ============================================================================

# Option 1: Databricks (default)
PROVIDER = "databricks"
AGENT_MODEL = "databricks-gpt-5"
JUDGE_MODEL = "databricks-gemini-2-5-flash"

# Option 2: OpenAI (uncomment to use)
# PROVIDER = "openai"
# AGENT_MODEL = "gpt-4o-mini"
# JUDGE_MODEL = "gpt-4o"

# Other settings
TEMPERATURE = 1.0
EXPERIMENT_NAME = "tool-selection-judge-notebook"

print(f"✓ Configuration set:")
print(f"  Provider: {PROVIDER}")
print(f"  Agent Model: {AGENT_MODEL}")
print(f"  Judge Model: {JUDGE_MODEL}")

✓ Configuration set:
  Provider: databricks
  Agent Model: databricks-gpt-5
  Judge Model: databricks-gemini-2-5-flash


### Verify Credentials (Optional Manual Setup)

If you didn't create a `.env` file, you can set credentials manually by uncommenting the appropriate lines below:

In [3]:
# ============================================================================
# MANUAL CREDENTIAL SETUP (if not using .env file)
# ============================================================================

# For Databricks - Uncomment and set these if you didn't create a .env file
# os.environ["DATABRICKS_TOKEN"] = "your-token-here"
# os.environ["DATABRICKS_HOST"] = "https://your-workspace.cloud.databricks.com"

# For OpenAI - Uncomment and set this if you didn't create a .env file
# os.environ["OPENAI_API_KEY"] = "sk-your-key-here"

# Verify credentials are set
if PROVIDER == "databricks":
    if "DATABRICKS_TOKEN" in os.environ and "DATABRICKS_HOST" in os.environ:
        print("✓ Databricks credentials found")
    else:
        print("⚠️  Missing Databricks credentials!")
        print("   Create a .env file or uncomment the lines above to set credentials")
elif PROVIDER == "openai":
    if "OPENAI_API_KEY" in os.environ:
        print("✓ OpenAI credentials found")
    else:
        print("⚠️  Missing OpenAI credentials!")
        print("   Create a .env file or uncomment the lines above to set credentials")

✓ Databricks credentials found


## Step 1: Setup MLflow Tracing

Enable MLflow tracing to capture all agent actions and LLM calls automatically.

In [4]:
from genai.common.mlflow_config import setup_mlflow_tracking

setup_mlflow_tracking(
    experiment_name=EXPERIMENT_NAME,
    enable_autolog=True
)

print("✓ MLflow tracing enabled")
print(f"  Experiment: {EXPERIMENT_NAME}")
print("  View traces: mlflow ui")

2025/11/28 13:31:11 INFO mlflow.tracking.fluent: Experiment with name 'tool-selection-judge-notebook' does not exist. Creating a new experiment.


✓ MLflow tracing enabled
  Experiment: tool-selection-judge-notebook
  View traces: mlflow ui


## Step 2: Define the Agent Class

This class demonstrates the complete LLM-as-a-Judge pattern:
1. Agent performs an action (`select_tool`) - traced with MLflow
2. Judge evaluates the action (`evaluate`) - uses `make_judge()`

In [ ]:
class AgentToolSelectionJudge:
    """
    Tutorial: LLM-as-a-Judge for Tool Selection Evaluation.
    
    The judge is a specialized LLM that assesses whether the agent made
    the right decision based on predefined criteria.
    """
    
    def __init__(self, config: AgentConfig, judge_model: str = None):
        """Initialize the agent and judge."""
        # Initialize the agent's LLM client
        provider_kwargs = config.get_provider_kwargs()
        self.client = get_client(config.provider, **provider_kwargs)
        self.config = config
        self.judge_model = judge_model or config.model
        
        # Initialize the MLflow judge
        self._init_judge()
    
    def _init_judge(self):
        """
        Create an MLflow Judge using make_judge().
        
        The judge:
        - Takes evaluation instructions (criteria)
        - Uses an LLM to perform the evaluation
        - Returns structured feedback (value + rationale)
        """
        # Set up environment for Databricks (needed by LiteLLM)
        if self.config.provider == "databricks":
            os.environ["OPENAI_API_KEY"] = os.environ.get("DATABRICKS_TOKEN", "")
            os.environ["OPENAI_API_BASE"] = f"{self.config.databricks_host}/serving-endpoints"
            model_uri = f"openai:/{self.judge_model}"
        else:
            model_uri = self.judge_model
        
        # Create the judge
        self.judge = make_judge(
            name="tool_selection_quality",
            instructions=get_judge_instructions(),
            feedback_value_type=Literal["correct", "incorrect"],
            model=model_uri
        )
    
    @mlflow.trace(span_type=SpanType.AGENT, name="select_tool")
    def select_tool(self, user_request: str, available_tools: List[str]) -> str:
        """
        Agent Action with MLflow Tracing.
        
        The @mlflow.trace decorator automatically captures:
        - Input parameters (user_request, available_tools)
        - Output (selected tool)
        - Execution time and metadata
        """
        # Get the tool selection prompt
        prompt = get_tool_selection_prompt(user_request, available_tools)
        
        # Call the LLM to select a tool
        api_params = {
            "model": self.config.model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 50
        }
        
        # Add temperature if supported (OpenAI only)
        if self.config.provider == "openai":
            api_params["temperature"] = 0.0
        
        response = self._call_llm(**api_params)
        tool_selected = response.choices[0].message.content.strip()
        
        # Note: No need to log_param here - the trace automatically captures inputs/outputs
        return tool_selected
    
    def evaluate(self, trace_id: str) -> Dict[str, Any]:
        """
        Evaluate with the Judge.
        
        The judge evaluates the trace and returns structured feedback:
        - feedback.value: "correct" or "incorrect"
        - feedback.rationale: Detailed explanation
        """
        # Fetch the trace from MLflow
        trace = mlflow.get_trace(trace_id)
        
        # Call the judge to evaluate the trace
        feedback = self.judge(trace=trace)
        
        # Return structured result
        return {
            "is_correct": feedback.value == "correct",
            "reasoning": feedback.rationale
        }
    
    @mlflow.trace(span_type=SpanType.LLM, name="llm_call")
    def _call_llm(self, **api_params):
        """Call LLM with MLflow tracing."""
        return self.client.chat.completions.create(**api_params)

## Step 3: Initialize Agent and Judge

Create the configuration and instantiate our judge.

In [6]:
# Create agent configuration
config = AgentConfig(
    model=AGENT_MODEL,
    provider=PROVIDER,
    temperature=TEMPERATURE
)

# Initialize the judge
judge = AgentToolSelectionJudge(config, judge_model=JUDGE_MODEL)

print("✓ Agent and Judge initialized")
print(f"  Provider: {config.provider}")
print(f"  Agent Model: {config.model}")
print(f"  Judge Model: {JUDGE_MODEL}")
print(f"  Temperature: {config.temperature}")

✓ Agent and Judge initialized
  Provider: databricks
  Agent Model: databricks-gpt-5
  Judge Model: databricks-gemini-2-5-flash
  Temperature: 1.0


## Step 4: Define Test Scenario

Set up a user query and available tools for the agent to choose from.

**💡 Try different queries to see how the agent and judge respond!**

In [7]:
# Define the scenario
user_request = "What's the weather like in San Francisco?"
available_tools = ["get_weather_api", "search_web", "get_calendar", "send_email"]

print("Test Scenario:")
print(f"  User Query: {user_request}")
print(f"  Available Tools: {available_tools}")

Test Scenario:
  User Query: What's the weather like in San Francisco?
  Available Tools: ['get_weather_api', 'search_web', 'get_calendar', 'send_email']


### 💡 Try Different Queries

Uncomment one of these examples or write your own:

In [ ]:
# Example queries to try:
#user_request = "Send email to John about the meeting"
# user_request = "What meetings do I have today?"
# user_request = "Search for information about machine learning"
# user_request = "What's the current stock price of AAPL?"

## Step 5: Agent Performs Action

The agent selects a tool based on the user query. MLflow automatically traces this action.

In [9]:
print("Agent is selecting a tool...\n")

tool_selected = judge.select_tool(user_request, available_tools)

print(f"✓ Agent selected: {tool_selected}")

Agent is selecting a tool...

✓ Agent selected: send_email


Trace(trace_id=tr-e77d6522597847039a2cedeb4aeedc13)

## Step 6: Judge Evaluates the Selection

Now the judge evaluates whether the agent made the right choice.

In [10]:
print("Judge is evaluating the selection...\n")

# Get the trace ID from the agent's action
trace_id = mlflow.get_last_active_trace_id()

# Evaluate with the judge
result = judge.evaluate(trace_id)

# Display results
print("=" * 70)
print(f"Decision: {'✓ CORRECT' if result['is_correct'] else '✗ INCORRECT'}")
print("\nReasoning:")
print(f"{result['reasoning']}")
print("=" * 70)

Judge is evaluating the selection...

Decision: ✓ CORRECT

Reasoning:
The user's request was "Send email to John about the meeting". The agent correctly selected the `send_email` tool from the available options: `get_weather_api`, `search_web`, `get_calendar`, `send_email`. This tool directly matches the user's intent and is the most suitable for the task. No more appropriate tools were overlooked.


[Trace(trace_id=tr-0c0313400682b9ee2822d18b02a37a1a), Trace(trace_id=tr-c303a3e08eb96efbdec656ffb39a972f), Trace(trace_id=tr-42d346827b20cc6cc3788eebf07477fa)]

## Step 7: View Detailed Traces

You can view detailed traces in the MLflow UI to see:
- The full conversation flow
- LLM inputs and outputs
- Execution times
- All logged parameters

Run this command in your terminal:
```bash
mlflow ui
```

Then navigate to: http://localhost:5000

## 🎯 Try It Yourself!

Run the complete workflow with a custom query:

In [11]:
def run_evaluation(query: str, tools: List[str]):
    """
    Complete workflow: Agent selects tool → Judge evaluates
    """
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"Available Tools: {tools}")
    print(f"{'='*70}\n")
    
    # Agent selects tool
    selected = judge.select_tool(query, tools)
    print(f"✓ Agent selected: {selected}\n")
    
    # Judge evaluates
    trace_id = mlflow.get_last_active_trace_id()
    result = judge.evaluate(trace_id)
    
    # Show results
    print(f"Decision: {'✓ CORRECT' if result['is_correct'] else '✗ INCORRECT'}")
    print(f"\nReasoning:\n{result['reasoning']}")
    print(f"\n{'='*70}\n")
    
    return result

# Try it!
result = run_evaluation(
    query="Send email to John about the meeting",
    tools=["get_weather_api", "search_web", "get_calendar", "send_email"]
)


Query: Send email to John about the meeting
Available Tools: ['get_weather_api', 'search_web', 'get_calendar', 'send_email']

✓ Agent selected: send_email

Decision: ✓ CORRECT

Reasoning:
The user's request was to "Send email to John about the meeting". The agent correctly identified and selected the `send_email` tool from the available options. This tool directly aligns with the user's intent and is the most appropriate choice among the given tools (`get_weather_api`, `search_web`, `get_calendar`).




[Trace(trace_id=tr-ed530ba71ed5014fe5504a9c77b59a7b), Trace(trace_id=tr-f5ae76d8e6d4f7533df73d02c3557bfe), Trace(trace_id=tr-d7d4f9e499b0bd377e7cf8e45fab1909), Trace(trace_id=tr-df42c3f40bda7ffe3414df78dab3d5a8)]

## 🧪 Experiment: Test Multiple Scenarios

Let's evaluate multiple queries and see how the judge performs:

In [ ]:
# Define test scenarios
test_scenarios = [
    ("What's the weather in Boston?", ["get_weather_api", "search_web", "get_calendar", "send_email"]),
    ("Schedule a meeting for tomorrow", ["get_weather_api", "search_web", "get_calendar", "send_email"]),
    ("Find information about Python", ["get_weather_api", "search_web", "get_calendar", "send_email"]),
    ("Send a message to Sarah", ["get_weather_api", "search_web", "get_calendar", "send_email"]),
]

# Run all scenarios
results = []
for query, tools in test_scenarios:
    result = run_evaluation(query, tools)
    results.append({
        "query": query,
        "correct": result["is_correct"]
    })

# Summary
correct_count = sum(1 for r in results if r["correct"])
total = len(results)
print(f"\n📊 Summary: {correct_count}/{total} selections were correct ({correct_count/total*100:.0f}%)")

## 🎨 Customization

### Modify Evaluation Criteria

The judge's evaluation criteria are defined in `prompts.py`. You can view them:

In [ ]:
print("Current Judge Instructions:")
print("=" * 70)
print(get_judge_instructions())
print("=" * 70)

### View Tool Selection Prompt

See how the agent is instructed to select tools:

In [ ]:
example_prompt = get_tool_selection_prompt(
    "What's the weather?",
    ["get_weather_api", "search_web"]
)

print("Example Tool Selection Prompt:")
print("=" * 70)
print(example_prompt)
print("=" * 70)

## 📚 Key Concepts Summary

### MLflow Tracing
- Automatically captures function inputs/outputs
- Tracks execution time and metadata
- Creates parent-child relationships for nested calls

### MLflow Judge
- Created with `make_judge()`
- Takes predefined evaluation criteria
- Returns structured feedback (value + rationale)

### Separation of Concerns
- **Agent**: Performs the task (tool selection)
- **Judge**: Evaluates the agent's performance
- **Prompts**: Define behavior (easy to modify in `prompts.py`)

## 🚀 Next Steps

1. **Explore MLflow UI**: Run `mlflow ui` to see detailed traces
2. **Modify Prompts**: Edit `prompts.py` to change evaluation criteria
3. **Try Different Models**: Experiment with different agent and judge models
4. **Add More Tools**: Expand the `available_tools` list
5. **Apply to Your Use Case**: Adapt this pattern for your own agent evaluations

## 📖 Resources

- [MLflow GenAI Judges Documentation](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html)
- [Original Tutorial](https://medium.com/@juanc.olamendy/using-llm-as-a-judge-to-evaluate-agent-outputs-a-comprehensive-tutorial-00b6f1f356cc)
- [Tool Selection Judge README](README.md)